In [23]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-large-mongolian-uncased', use_fast=False)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-large-mongolian-uncased')

# tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-base-mongolian-uncased', use_fast=False)
# model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-base-mongolian-uncased')

## declare task ##
pipe = pipeline(task="fill-mask", model=model, tokenizer=tokenizer)

import pickle
with open('mask.pickle', 'rb') as f:
    MASK = pickle.load(f)



tokenizer_config.json:   0%|          | 0.00/45.0 [00:00<?, ?B/s]

c:\Users\quezi\.conda\envs\weibodatacleaning\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\quezi\.cache\huggingface\hub\models--tugstugi--bert-large-mongolian-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

In [21]:
for i in range(len(temp['sen'])):
    input_ = temp['sen'].copy()
    input_[i] = '[MASK]'
    input_ = " ".join(input_)
    output_ = pipe(input_)
    for j in output_:
        print(j)
    print(input_)

{'score': 0.28184500336647034, 'token': 23406, 'token_str': 'Үлгэр', 'sequence': 'Үлгэр домог үлгэр худал биш, балар эртний тухай бодит үнэн оршиж байдаг.'}
{'score': 0.08691340684890747, 'token': 446, 'token_str': 'Гэвч', 'sequence': 'Гэвч домог үлгэр худал биш, балар эртний тухай бодит үнэн оршиж байдаг.'}
{'score': 0.08220097422599792, 'token': 200, 'token_str': 'Гэхдээ', 'sequence': 'Гэхдээ домог үлгэр худал биш, балар эртний тухай бодит үнэн оршиж байдаг.'}
{'score': 0.0479096956551075, 'token': 11620, 'token_str': 'Эртний', 'sequence': 'Эртний домог үлгэр худал биш, балар эртний тухай бодит үнэн оршиж байдаг.'}
{'score': 0.04697900265455246, 'token': 83, 'token_str': 'Харин', 'sequence': 'Харин домог үлгэр худал биш, балар эртний тухай бодит үнэн оршиж байдаг.'}
[MASK] домог үлгэр худал биш , балар эртний тухай бодит үнэн оршиж байдаг .
{'score': 0.0753706693649292, 'token': 1559, 'token_str': 'тэнд', 'sequence': 'тэгэхлээр тэнд үлгэр худал биш, балар эртний тухай бодит үнэн орши

In [17]:
temp = mask[21]
maskindex = 0
for i in range(len(temp['sen'])):
    if temp['sen'][i] == '<>':
        temp['sen'][i] = temp['word'][maskindex*2]
        maskindex += 1
temp

{'sen': ['тэгэхлээр',
  'домог',
  'үлгэр',
  'худал',
  'биш',
  ',',
  'балар',
  'эртний',
  'тухай',
  'бодит',
  'үнэн',
  'оршиж',
  'байдаг',
  '.'],
 'word': ['үнэн', 'нүэн']}

In [ ]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/albert-mongolian")
model = AutoModelForMaskedLM.from_pretrained("bayartsogt/albert-mongolian")

########################################################################################################

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/mongolian-roberta-large")
model = AutoModelForMaskedLM.from_pretrained("bayartsogt/mongolian-roberta-large")




